## Importing Stuff

In [1]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

## Getting the Data Set

In [2]:
ames_data = pd.read_csv("data/AmesHousing.csv")

In [3]:
ames_data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [4]:
ames_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order            2930 non-null   int64  
 1   PID              2930 non-null   int64  
 2   MS SubClass      2930 non-null   int64  
 3   MS Zoning        2930 non-null   object 
 4   Lot Frontage     2440 non-null   float64
 5   Lot Area         2930 non-null   int64  
 6   Street           2930 non-null   object 
 7   Alley            198 non-null    object 
 8   Lot Shape        2930 non-null   object 
 9   Land Contour     2930 non-null   object 
 10  Utilities        2930 non-null   object 
 11  Lot Config       2930 non-null   object 
 12  Land Slope       2930 non-null   object 
 13  Neighborhood     2930 non-null   object 
 14  Condition 1      2930 non-null   object 
 15  Condition 2      2930 non-null   object 
 16  Bldg Type        2930 non-null   object 
 17  House Style   

Initial Impression: There are many features with LOTS of missing values. For example, "Alley" has well over 90% of its values as NaN. Let's see how many of these have lots of NaN values. We'll start by making a dictionary that tells us the number of NaN values for each column. 

In [5]:
null_count_dict = {col_name: ames_data[col_name].isna().sum() for col_name in ames_data.columns}

Let's test the dictionary. Below, the "Alley" column should have the number of NaN values. There are 2930 entries and 198 of them are not NaN. 

In [6]:
null_count_dict["Alley"]

2732

It works! 

Now, after playing around with the "Alley" and "Street" columns, it seems doubtful that this information helps at all. I guess it could, somehow, but I say it seems doubtful because out of the few times when Alley isn't NaN, 120 of those are "Grvl" while Street is "Pave". The rest (78, I believe) just match. So unless there is significance in these 120 identical mismatches, we should just drop the Alley column altogether. 

In [7]:
ames_data.dropna(subset="Alley")[ames_data["Alley"] != ames_data["Street"]][["Alley", "Street"]].value_counts()

C:\Users\natha\AppData\Local\Temp\ipykernel_8544\2599212873.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ames_data.dropna(subset="Alley")[ames_data["Alley"] != ames_data["Street"]][["Alley", "Street"]].value_counts()


Alley  Street
Grvl   Pave      120
Name: count, dtype: int64

Now let's hunt for other columns like "Alley" that have lots of null values. 

In [8]:
sorted_list = sorted(list(null_count_dict.items()), key=lambda x:x[1], reverse=True)
for key, value in sorted_list:
    print(key, value)

Pool QC 2917
Misc Feature 2824
Alley 2732
Fence 2358
Mas Vnr Type 1775
Fireplace Qu 1422
Lot Frontage 490
Garage Yr Blt 159
Garage Finish 159
Garage Qual 159
Garage Cond 159
Garage Type 157
Bsmt Exposure 83
BsmtFin Type 2 81
Bsmt Qual 80
Bsmt Cond 80
BsmtFin Type 1 80
Mas Vnr Area 23
Bsmt Full Bath 2
Bsmt Half Bath 2
BsmtFin SF 1 1
BsmtFin SF 2 1
Bsmt Unf SF 1
Total Bsmt SF 1
Electrical 1
Garage Cars 1
Garage Area 1
Order 0
PID 0
MS SubClass 0
MS Zoning 0
Lot Area 0
Street 0
Lot Shape 0
Land Contour 0
Utilities 0
Lot Config 0
Land Slope 0
Neighborhood 0
Condition 1 0
Condition 2 0
Bldg Type 0
House Style 0
Overall Qual 0
Overall Cond 0
Year Built 0
Year Remod/Add 0
Roof Style 0
Roof Matl 0
Exterior 1st 0
Exterior 2nd 0
Exter Qual 0
Exter Cond 0
Foundation 0
Heating 0
Heating QC 0
Central Air 0
1st Flr SF 0
2nd Flr SF 0
Low Qual Fin SF 0
Gr Liv Area 0
Full Bath 0
Half Bath 0
Bedroom AbvGr 0
Kitchen AbvGr 0
Kitchen Qual 0
TotRms AbvGrd 0
Functional 0
Fireplaces 0
Paved Drive 0
Wood Deck 

Now we have some other columns that are likely in the same boat as Alley. It might be a good idea to just drop the columns we see through Fireplace Qu, since they all seem to have half or more of their values missing. If we're curious, there's always the option of coming back and seeing if using these columns would be helpful. 

In [9]:
slimmer_df = ames_data.drop(columns=["Pool QC", "Misc Feature", "Alley", "Fence", "Mas Vnr Type", "Fireplace Qu"]).dropna()
slimmer_df.shape

(2218, 76)

On second thought, let's drop more columns. I'd like to get to the machine learning part. So we'll drop the columns through Mass Vnr Area, which has 23 missing values. All the later columns have at most 2. And since there are 18 columns now, I don't feel like typing them all out. So let's use this instead: 

In [13]:
# Confirm that "Mas Vnr Area" is the place where we stop, i.e. index 17
print(sorted_list[17][0])
# Creating a new df without NaN values: The first 18 columns are dropped. The index is reset, and the old "index" column called "Order" is dropped. 
cleaned_df = ames_data.drop(columns=[key for key, value in sorted_list[:18]]).dropna(how="any").reset_index(drop=True).drop(columns=["Order"]) #for dropna, "any" means that rows with "any" NaN values will be dropped. 
# Check number of rows and columns left in the dataframe
print("Rows and Columns:", cleaned_df.shape)

# Save to a new CSV
cleaned_df.to_csv("data/CleanedAmesHousing.csv")

Mas Vnr Area
Rows and Columns: (2926, 63)


Okay, highly NaN-valued columns dropped! If we only drop columns through the previously-mentioned slimmer dataframe, then we could focus on imputations. Something in me just didn't want to do that now because I want to do some machine learning. 